<h5 style="color:rgba(0, 0, 178, 0.6);">This code hidden warrnings</h5>

In [1]:
import warnings
warnings.filterwarnings('ignore')

<h5 style="color:rgba(0, 0, 178, 0.6);">Install librarys that we need</h5>

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("bmh")
from tensorflow import keras
from sklearn.cluster import KMeans
import random
import tensorflow as tf
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
tf.__version__

'2.16.1'

In [3]:
# read the orginal data from the csv file
df1 = pd.read_csv("well2.csv")
df1['date'] = pd.to_datetime(df1['date'], errors='coerce') #convert date column to datetime format
# =========================================
# create a dummy date time with freq = 1D to see which days we have no data
df = pd.DataFrame({
    'date': pd.date_range(start='2013-07-24', periods=1152, freq='D')
})
# =========================================
# Merge on the 'date' column, using 'left' join to keep all dates from df1
df = df.merge(df1, on='date', how='left')

df = df[[
    "date", "bhp", "bht",
    "dp_tubing", "AVG_CHOKE_SIZE_P",
    "QG"]]
df.set_index('date', inplace=True)
df['Time'] = np.arange(len(df.index))
df.dropna(inplace=True)
df.head()

,bhp,bht,dp_tubing,AVG_CHOKE_SIZE_P,QG,Time
date,,,,,,
2013-07-24,1.926270,0.35220,26.123040,3.256548,9505.611429,0
2013-07-25,163.594260,60.31574,61.473080,8.549131,108940.110000,1
2013-07-28,0.000000,0.00000,0.000000,2.464222,257479.142857,4
2013-07-29,0.000000,0.00000,0.000000,2.536987,112755.000000,5
2013-07-31,273.946931,105.55137,179.382345,2.540804,137705.269214,7


In [6]:
def make_lags(ts, lags, lead_time=1):
    return pd.concat(
        {
            f'y_lag_{i}': ts.shift(i)
            for i in range(lead_time, lags + lead_time)
        },
        axis=1)

def make_roling_lag(df, name="QG"):
    return pd.concat(
        [
            df.shift(1).rolling(7).mean().rename(f"{name}_lagged_mean_7D"),
            df.shift(1).rolling(7).max().rename(f"{name}_lagged_max_7D"),
            df.shift(1).rolling(7).min().rename(f"{name}_lagged_min_7D"),
            df.shift(1).rolling(15).mean().rename(f"{name}_lagged_mean_15D"),
            df.shift(1).rolling(15).max().rename(f"{name}_lagged_max_15D"),
            df.shift(1).rolling(15).min().rename(f"{name}_lagged_min_15D"),
            df.shift(1).rolling(30).mean().rename(f"{name}_lagged_mean_30D"),
            df.shift(1).rolling(30).max().rename(f"{name}_lagged_max_30D"),
            df.shift(1).rolling(30).min().rename(f"{name}_lagged_min_30D"),
        ],
        axis="columns"
    )

def make_multistep_target(ts, steps):
    return pd.concat(
        {f'y_step_{i + 1}': ts.shift(-i)
         for i in range(steps)},
        axis=1)


In [5]:
search_terms = ["bhp", "bht", "dp_tubing", "AVG_CHOKE_SIZE_P", "Time"]
X0=make_lags(df[search_terms], lags=4)
X0.columns = [' '.join(col).strip() for col in X0.columns.values]

search_terms = ["bhp", "bht", "dp_tubing", "AVG_CHOKE_SIZE_P", "Time", "QG"]



lags=make_lags(df[search_terms], lags=7)
lags.columns = [' '.join(col).strip() for col in lags.columns.values]
lagged_df = pd.concat(
    [
        df,
        lags,
        make_roling_lag(df["QG"], name="QG"),
        make_roling_lag(df["bhp"], name="bhp"),
        make_roling_lag(df["bht"], name="bht"),
        make_roling_lag(df["dp_tubing"], name="dp_tubing"),
        make_roling_lag(df["AVG_CHOKE_SIZE_P"], name="AVG_CHOKE_SIZE_P")
    ],
    axis="columns",
)
X = lagged_df.copy().fillna(0.0)
X

,bhp,bht,dp_tubing,AVG_CHOKE_SIZE_P,QG,Time,y_lag_1 bhp,y_lag_1 bht,y_lag_1 dp_tubing,y_lag_1 AVG_CHOKE_SIZE_P,...,dp_tubing_lagged_min_30D,AVG_CHOKE_SIZE_P_lagged_mean_7D,AVG_CHOKE_SIZE_P_lagged_max_7D,AVG_CHOKE_SIZE_P_lagged_min_7D,AVG_CHOKE_SIZE_P_lagged_mean_15D,AVG_CHOKE_SIZE_P_lagged_max_15D,AVG_CHOKE_SIZE_P_lagged_min_15D,AVG_CHOKE_SIZE_P_lagged_mean_30D,AVG_CHOKE_SIZE_P_lagged_max_30D,AVG_CHOKE_SIZE_P_lagged_min_30D
date,,,,,,,,,,,,,,,,,,,,,
2013-07-24,1.926270,0.352200,26.123040,3.256548,9505.611429,0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
2013-07-25,163.594260,60.315740,61.473080,8.549131,108940.110000,1,1.926270,0.352200,26.123040,3.256548,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
2013-07-28,0.000000,0.000000,0.000000,2.464222,257479.142857,4,163.594260,60.315740,61.473080,8.549131,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
2013-07-29,0.000000,0.000000,0.000000,2.536987,112755.000000,5,0.000000,0.000000,0.000000,2.464222,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
2013-07-31,273.946931,105.551370,179.382345,2.540804,137705.269214,7,0.000000,0.000000,0.000000,2.536987,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-09-13,253.399422,105.266981,226.400814,100.000000,88042.160000,1147,253.658108,105.259036,226.680342,100.000000,...,223.897904,100.0,100.0,100.0,99.722222,100.0,97.916667,99.368745,100.0,97.916667
2016-09-14,253.209915,105.271539,226.255384,100.000000,90188.970000,1148,253.399422,105.266981,226.400814,100.000000,...,223.897904,100.0,100.0,100.0,99.722222,100.0,97.916667,99.420539,100.0,97.916667
2016-09-15,252.938710,105.274111,226.036656,100.000000,91890.070000,1149,253.209915,105.271539,226.255384,100.000000,...,223.897904,100.0,100.0,100.0,99.722222,100.0,97.916667,99.472771,100.0,97.916667


In [7]:
# Eight-week forecast
y = make_multistep_target(df["QG"], steps=10).dropna()

# Shifting has created indexes that don't match. Only keep times for
# which we have both targets and features.
y, X = y.align(X, join='inner', axis=0)
y

,y_step_1,y_step_2,y_step_3,y_step_4,y_step_5,y_step_6,y_step_7,y_step_8,y_step_9,y_step_10
date,,,,,,,,,,
2013-07-24,9505.611429,108940.110000,257479.142857,112755.000000,137705.269214,141268.448264,161227.00,160270.49,160951.30,160232.470000
2013-07-25,108940.110000,257479.142857,112755.000000,137705.269214,141268.448264,161227.000000,160270.49,160951.30,160232.47,159483.760000
2013-07-28,257479.142857,112755.000000,137705.269214,141268.448264,161227.000000,160270.490000,160951.30,160232.47,159483.76,162196.780000
2013-07-29,112755.000000,137705.269214,141268.448264,161227.000000,160270.490000,160951.300000,160232.47,159483.76,162196.78,161999.010000
2013-07-31,137705.269214,141268.448264,161227.000000,160270.490000,160951.300000,160232.470000,159483.76,162196.78,161999.01,160095.110000
...,...,...,...,...,...,...,...,...,...,...
2016-09-04,87504.870000,87798.790000,88650.380000,88563.480000,89379.120000,89482.000000,90470.31,90540.97,90810.01,88042.160000
2016-09-05,87798.790000,88650.380000,88563.480000,89379.120000,89482.000000,90470.310000,90540.97,90810.01,88042.16,90188.970000
2016-09-06,88650.380000,88563.480000,89379.120000,89482.000000,90470.310000,90540.970000,90810.01,88042.16,90188.97,91890.070000


In [10]:
# Normalize features 
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
scaled_X = scaler_X.fit_transform(X.values)
series_X = pd.DataFrame(scaled_X, columns=X.columns)
scaled_y = scaler_y.fit_transform(y.values)
series_y = pd.DataFrame(scaled_y, columns=y.columns)
print(series_y.head())
print(series_y.shape)

   y_step_1  y_step_2  y_step_3  y_step_4  y_step_5  y_step_6  y_step_7  \
0  0.000000  0.256852  0.726162  0.268906  0.347736  0.358994  0.422053   
1  0.297135  0.726162  0.268906  0.347736  0.358994  0.422053  0.419031   
2  0.741006  0.268906  0.347736  0.358994  0.422053  0.419031  0.421182   
3  0.308534  0.347736  0.358994  0.422053  0.419031  0.421182  0.418911   
4  0.383092  0.358994  0.422053  0.419031  0.421182  0.418911  0.416545   

   y_step_8  y_step_9  y_step_10  
0  0.419031  0.421182   0.418911  
1  0.421182  0.418911   0.416545  
2  0.418911  0.416545   0.425117  
3  0.416545  0.425117   0.424492  
4  0.425117  0.424492   0.418477  
(1110, 10)


In [18]:
# Split data into training (70%) and test (30%)
nrow = round(0.7*X.shape[0])
X_train = series_X.iloc[:nrow, :]
y_train = series_y[:nrow]
X_test = series_X.iloc[nrow:,:]
y_test = series_y[nrow:]

In [19]:
# X_train = X_train.values.reshape(X_train.shape[0],X_train.shape[1],1)
# X_test = X_test.values.reshape(X_test.shape[0],X_test.shape[1],1)
# print(X_train.shape)
# print(X_test.shape)

<h5 style="color:rgba(0, 0, 178, 0.6);">Create a function that build a dynamic model that can change the shape of the NN model with hyper parameters(hp) parameters</h5>

In [36]:
# hp ======> hyper parameters = [number_nuron_layer1, number_nuron_layer2, number_nuron_layer3,
#                          number_nuron_layer4, Activation_fuctin_layer1, Activaiton_function_layer2,
#                     Activation_fuctin_layer3, Activation_fuctin_layer4, If for layer1 have Drop_out=1 else 0,
#         If for layer2 have Drop_out=1 else 0, If for layer3 have Drop_out=1 else 0, If for layer4 have Drop_out=1 else 0,
#     If for layer1 have batch_norm=1 else 0, If for layer2 have batch_norm=1 else 0, If for layer1 have batch_norm=3 else 0,
#                       If for layer1 have batch_norm=1 else 0, learning_rate, batch_size]


# in this function we create a function that can build model that can chane evry time

def dynamic_model(hp, train=X_train):


    activation_function = ["relu","sigmoid","softmax","softsign",
                           "tanh","selu","elu" ,"exponential",
                           "leaky_relu","relu6","silu","gelu",
                           "hard_sigmoid","linear","mish","log_softmax"]
    

    input_layer = keras.layers.Input(shape=(93,))
    layer1 = keras.layers.Dense([93, hp[0]], activation_function[int(hp[4])], name="layer1")(input_layer)
    
#     in this sectino we have 4 layer  so that we have 4 section for dscetion that we have in each layer DropOut or BatchNorm
#   this is layer1 we have 4 condition:
    if ((hp[8] == 1) & (hp[12] == 1)):
        drop1 = keras.layers.Dropout(0.3)(layer1)
        norm1 = keras.layers.BatchNormalization()(drop1)
        layer2 = keras.layers.Dense(hp[1], activation_function[int(hp[5])], name="layer2")(norm1)
    elif ((hp[8] == 1) & (hp[12] == 0)):
        drop1 = keras.layers.Dropout(0.3)(layer1)
        layer2 = keras.layers.Dense(hp[1], activation_function[int(hp[5])], name="layer2")(drop1)
    elif ((hp[8] == 0) & (hp[12] == 1)):
        norm1 = keras.layers.BatchNormalization()(layer1)
        layer2 = keras.layers.Dense(hp[1], activation_function[int(hp[5])], name="layer2")(norm1)
    else:
        layer2 = keras.layers.Dense(hp[1], activation_function[int(hp[5])], name="layer2")(layer1)
        
#    layer2:     
    if ((hp[9] == 1) & (hp[13] == 1)):
        drop2 = keras.layers.Dropout(0.3)(layer2)
        norm2 = keras.layers.BatchNormalization()(drop2)
        layer3 = keras.layers.Dense(hp[2], activation_function[int(hp[6])], name="layer3")(norm2)
    elif ((hp[9] == 1) & (hp[13] == 0)):
        drop2 = keras.layers.Dropout(0.3)(layer2)
        layer3 = keras.layers.Dense(hp[2], activation_function[int(hp[6])], name="layer3")(drop2)
    elif ((hp[9] == 0) & (hp[13] == 1)):
        norm2 = keras.layers.BatchNormalization()(layer2)
        layer3 = keras.layers.Dense(hp[2], activation_function[int(hp[6])], name="layer3")(norm2)
    else:
        layer3 = keras.layers.Dense(hp[2], activation_function[int(hp[6])], name="layer3")(layer2)
        
#     layer3 :         
    if ((hp[10] == 1) & (hp[14] == 1)):
        drop3 = keras.layers.Dropout(0.3)(layer3)
        norm3 = keras.layers.BatchNormalization()(drop3)
        layer4 = keras.layers.Dense(hp[3], activation_function[int(hp[7])], name="layer4")(norm3)
    elif ((hp[10] == 1) & (hp[14] == 0)):
        drop3 = keras.layers.Dropout(0.3)(layer3)
        layer4 = keras.layers.Dense(hp[3], activation_function[int(hp[7])], name="layer4")(drop3)
    elif ((hp[10] == 0) & (hp[14] == 1)):
        norm3 = keras.layers.BatchNormalization()(layer3)
        layer4 = keras.layers.Dense(hp[3], activation_function[int(hp[7])], name="layer4")(norm3)
    else:
        layer4 = keras.layers.Dense(hp[3], activation_function[int(hp[7])], name="layer4")(layer3)
        
#    layer4:        
    if ((hp[11] == 1) & (hp[15] == 1)):
        drop4 = keras.layers.Dropout(0.3)(layer4)
        norm4 = keras.layers.BatchNormalization()(drop4)
        output_layer = keras.layers.Dense(1, "linear", name="output_layer")(norm4)
    elif ((hp[11] == 1) & (hp[15] == 0)):
        drop4 = keras.layers.Dropout(0.3)(layer4)
        output_layer = keras.layers.Dense(1, "linear", name="output_layer")(drop4)
    elif ((hp[11] == 0) & (hp[15] == 1)):
        norm4 = keras.layers.BatchNormalization()(layer4)
        output_layer = keras.layers.Dense(1, "linear", name="output_layer")(norm4)
    else:
        output_layer = keras.layers.Dense(1, "linear", name="output_layer")(layer4)

#     at last we create final model        
    model = keras.Model(inputs=input_layer, outputs=output_layer)
    
#     in this section we complie the model    
    optimizer =keras.optimizers.Adam(learning_rate=hp[16])
    model.compile(loss="mae", optimizer=optimizer, metrics=["mse"])

    # wright the callbacks that we need
#     checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_checkpoints.h5", save_weights_only=True,best_only=True)
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)
    
    # fit the model
    history = model.fit(X_train,
                         y_train, 
                        validation_data=(X_test, y_test),
                        callbacks=[early_stopping_cb],
                        batch_size=int(hp[17]),
                        epochs=1000,
                        verbose=0)
    
#     in this section we complie the model
    val_pred = model.predict(X_test)
#   because we r2_score dont work with nan values we chane nan value with -999
    val_pred[np.isnan(val_pred)] = -999
        
    solution_fitness = r2_score(y_test, val_pred)
    print(solution_fitness)
 
    return solution_fitness


In [37]:
def hp_creator(num_solutions=100):
    hps = pd.DataFrame(columns=[
                                "nurn1","nurn2","nurn3","nurn4",
                                "Active1","Active2","Active3","Active4",
                                "Drop1", "Drop2", "Drop3", "Drop4",
                                "Norm1", "Norm2", "Norm3", "Norm4",
                                "Learn_rate", "batch_size"
                               ])
    
    hps["nurn1"], hps["nurn2"] = [random.randint(10, 200) for _ in range(num_solutions)], [random.randint(10, 200) for _ in range(num_solutions)]
    hps["nurn3"], hps["nurn4"] = [random.randint(10, 200) for _ in range(num_solutions)], [random.randint(10, 200) for _ in range(num_solutions)]
    
    hps["Active1"], hps["Active2"] = [random.randint(0, 15) for _ in range(num_solutions)], [random.randint(0, 15) for _ in range(num_solutions)]
    hps["Active3"], hps["Active4"] = [random.randint(0, 15) for _ in range(num_solutions)], [random.randint(0, 15) for _ in range(num_solutions)]
                                                                                 
    hps["Drop1"], hps["Drop2"] = [random.choice([0, 1]) for _ in range(num_solutions)], [random.choice([0, 1]) for _ in range(num_solutions)]
    hps["Drop3"], hps["Drop4"] = [random.choice([0, 1]) for _ in range(num_solutions)], [random.choice([0, 1]) for _ in range(num_solutions)]
    
    hps["Norm1"], hps["Norm2"] = [random.choice([0, 1]) for _ in range(num_solutions)], [random.choice([0, 1]) for _ in range(num_solutions)]
    hps["Norm3"], hps["Norm4"] = [random.choice([0, 1]) for _ in range(num_solutions)], [random.choice([0, 1]) for _ in range(num_solutions)]
    
    hps["Learn_rate"] = [random.choice([0.001, 0.01, 0.0001, 0.00001, 0.1, 0.000001, 0.0000001]) for _ in range(num_solutions)]
    hps["batch_size"] = [random.randint(30, 300) for _ in range(num_solutions)]
    

    return hps

In [38]:
# num_particles ==> number of rnadom input that generate
# max_iter ==> number of iteration that function use to solve the problem
# w and c1 and c2 ==> constant parameters this are hayper parameter


def pso(num_particles=100, max_iter=20, w=0.5, c1=1, c2=2):
    # Initialize particles and velocities
    particles = hp_creator(num_solutions=num_particles).values
    velocities = np.zeros((num_particles, len(particles[0,:])))

    # Initialize the best positions and fitness values
    best_positions = np.copy(particles)
    best_fitness = np.array([dynamic_model(p) for p in particles])
    swarm_best_position = best_positions[np.argmax(best_fitness)]
    swarm_best_fitness = np.max(best_fitness)
    
    # Iterate through the specified number of iterations, updating the velocity and position of each particle at each iteration
    for k in range(max_iter):
        # Update velocities
        r1 = np.random.uniform(0, 1, (num_particles, 18))
        r2 = np.random.uniform(0, 1, (num_particles, 18))
        velocities = w * velocities + c1 * r1 * (best_positions - particles) + c2 * r2 * (swarm_best_position - particles)

        # Update positions
        particles += velocities
# --------------------------------------------------------------------------------------------------------------
#       in this section we need to the updated positions in bonderys that we need so we set ruls to it
#       ferst for for check each particle and secend for to check each parameters
        for i, hp in enumerate(particles):
            for j, param in enumerate(hp):
                #------------------------------------------------------------------------------------------------
                #bounders for number of nurouns in each leayer
                if (j>=0)&(j<4):
                    while (10>particles[i,j])or(200<particles[i,j]):
                        if particles[i,j]<10:
                            particles[i,j] = 10 + (10-int(particles[i,j]))
                        elif particles[i,j]>200:
                            particles[i,j] = 200-(int(particles[i,j])-200)
                    particles[i,j] = int(particles[i,j])
               #--------------------------------------------------------------------------------------------------
                #bounders for which activation we should use    
                if (j>=4)&(j<8):
                    while (0>particles[i,j])or(15<particles[i,j]):
                        if particles[i,j]<0:
                            particles[i,j] = 0 - int(round(particles[i,j]))
                        elif particles[i,j]>15:
                            particles[i,j] = 15-(int(round(particles[i,j])-15))
                    particles[i,j] = int(round(particles[i,j]))
              #----------------------------------------------------------------------------------------------------
                #bounders for if we have Dropuout or Batchnormalize or not
                if (j>=8)&(j<16):
                    if particles[i,j]<=0.5:
                        particles[i,j] = 0
                    elif particles[i,j]>0.5:
                        particles[i,j] = 1
             #------------------------------------------------------------------------------------------------------
                #bounders for learning rate
                if j==16:
                    while (10e-7>particles[i,j])or(0.1<particles[i,j]):
                        if particles[i,j]<10e-7:
                            particles[i,j] = 10e-7 + (10e-7-particles[i,j])
                        elif particles[i,j]>0.1:
                            particles[i,j] = 0.1-(particles[i,j]-0.1)
            #------------------------------------------------------------------------------------------------------
                #bounders for batch size
                if j==17:
                    while (30>particles[i,j])or(300<particles[i,j]):
                        if particles[i,j]<30:
                            particles[i,j] = 30 + (30-int(particles[i,j]))
                        elif particles[i,j]>300:
                            particles[i,j] = 300-(int(particles[i,j])-300)
                    particles[i,j] = int(particles[i,j])
#------------------------------------------------------------------------------------------------------------------
                    
        # Evaluate fitness of each particle
        print("=========================================================================")
        print("iter:",k)
        fitness_values = np.array([dynamic_model(p) for p in particles])
        print("==========================================================================")
        # Update best positions and fitness values
        improved_indices = np.where(fitness_values > best_fitness)
        best_positions[improved_indices] = particles[improved_indices]
        best_fitness[improved_indices] = fitness_values[improved_indices]
        if np.max(fitness_values) > swarm_best_fitness:
            swarm_best_position = particles[np.argmax(fitness_values)]
            swarm_best_fitness = np.max(fitness_values)

    # Return the best solution found by the PSO algorithm
    return swarm_best_position, swarm_best_fitness


# Run the PSO algorithm on the Rastrigin function
solution, fitness = pso(max_iter=5)

# Print the solution and fitness value
print('Solution:', solution)
print('Fitness:', fitness)

ValueError: Invalid dtype: TrackedList